In [47]:
# Replace content in about.html

fname = '../public_html/About.html'
label = '<!-- content label -->\n'
file = open(fname,'r',encoding='utf-8')
switch = 0
tmp = 0
segment = {}
seg_label = 0
once = 1
name = []
for line in file.readlines():
    tmp = tmp + 1
    if switch%2==0:
        if once==1:
            seg_label = seg_label + 1
            name.append("seg"+str(seg_label))
            segment[name[seg_label-1]] = []
            if switch!=0:
                segment[name[seg_label-1]].append(label)
            once = 0
        segment[name[seg_label-1]].append(line)
    if line==label:
        switch = switch +1
        once = 1

cname = '../public_html/markdown-html/personal_statement.html'
content = open(cname,'r',encoding='utf-8').readlines()
new = segment[name[0]]
new.extend(content[1:-1])
new.extend(segment[name[1]])

file.close()

file = open(fname, 'w', encoding='utf-8')
file.writelines(new)

file.close()

In [166]:
# Replace and configure avatar in About.html

from PIL import Image
# img.save('img.jpg', quality=50) --> quality: 1(worst)~95(best) decides compression level

img_path = '../public_html/bran.jpg'
img = Image.open(img_path)

# configure image size and crop it to round form here --> using tkinter and PIL.ImageTK, etc to create a simple GUI

fpath = '../public_html/Chang_files/ian.png'
img.save(fpath, quality=50)

In [55]:
# Refresh items in Learning.html

from os import listdir
import re

def findPositions(line):  # find positions of > extraction <
    posi_ini = line.find('<')
    posi1 = line.find('>') + 1
    posi2 = line[posi_ini+1:].find('<') + posi_ini + 1
    return (posi1, posi2)

def excerptFrom(item):  # create excerpt
    item_path = '../public_html/LEARNING/' + item + '.html'
    lines = open(item_path, 'r', encoding='utf-8').readlines()
    tmp = 0
    label = []
    cnt = 0
    for line in lines:
        tmp = tmp + 1
        if 'content label' in line:
            cnt = cnt + 1
            label.append(tmp)
    content = lines[label[0]:label[1]]
    string = ''
    for line in content:
        while re.search('<.*?>',line):  # string regular expression -> find format '< >'
            line = line.replace(re.search('<.*?>',line).group(0),'')
        string = string + line
    return (string[:500], cnt/2)  # length of excerption & number of pages in current catogory

fpath = '../public_html/Learning.html'
label_title = 'LEARNING/'
label_content = 'learning-excerpt'
file = open(fpath,'r',encoding='utf-8').readlines()

# traversal -> list current items
segment = {}
parts = {}
title = []
content = []
line_label = []
tmp = 0
for line in file:
    tmp = tmp + 1
    if label_title in line:
        line_label.append(tmp)
        [posi1,posi2] = findPositions(line)
        title.append(line[posi1:posi2])
    if label_content in line:
        [posi1,posi2] = findPositions(file[tmp])
        content.append(file[tmp][posi1:posi2])

for i in range(len(title)):
    segment[title[i]] = content[i]
    parts[title[i]] = file[line_label[i]-2:line_label[i]+4]

title_format = file[line_label[0]-4:line_label[0]+1]
content_format = file[line_label[0]+1:line_label[0]+6]

# refreshing  -->  check if any update exists in LEARNING directory
learning_path = '../public_html/LEARNING'
files = listdir(learning_path)

for i in title:    # remove extra
    if i+'.html' not in files:
        segment.pop(i)
        parts.pop(i)
        
for i in range(len(files)):    # refresh
    item = files[i][:-5]
    if item not in title:
        title.append(item)
    [excerpt, pnum] = excerptFrom(item)
    excerpt = excerpt[:excerpt.rfind(' ')].strip() + ' ...'
    excerpt = excerpt.replace('\n',' ')
    title_nw = title_format[:]
    content_nw = content_format[:]
    posi_ini = title_nw[3].find('<')
    title_nw[3] = title_nw[3][:posi_ini] + '<a href="LEARNING/' + item + '">' + item + '(' + str(int(pnum)) + ')' + '</a>\n'
    [posi1,posi2] = findPositions(content_nw[1])
    segment[item] = excerpt
    content_nw[1] = content_nw[1][:posi1] + excerpt + content_nw[1][posi2:]
    title_nw.extend(content_nw)
    parts[item] = title_nw

head = file[:line_label[0]-4]
foot = file[line_label[-1]+6:]
body = []

for i in sorted(segment, key=str.casefold):
    body.extend(parts[i])

head.extend(body)
head.extend(foot)
path_test = '../public_html/learning.html'
open(path_test, 'w', encoding='utf-8').writelines(head)

In [78]:
# add/delete or update learning directory
# need refresh button to make sure files are up to date

import os
import random
from datetime import date
from datetime import datetime

def findPositions(line):
    posi_ini = line.find('<')
    posi1 = line.find('>') + 1
    posi2 = line[posi_ini+1:].find('<') + posi_ini + 1
    return (posi1, posi2)

def sortbyDate(html):
    tmp = 0
    astruct = {}
    dates = []
    atmp = []
    anw = []
    html_nw = []
    for line in html:
        tmp = tmp + 1
        if 'article label' in line:
            atmp.append(tmp-1)
        if 'time datetime' in line:
            posi = findPositions(line)
            dates_str = line[posi[0]:posi[1]]
            ptime = datetime.strptime(dates_str, "%B %d, %Y").timestamp()
            if ptime in dates:
                ptime = ptime + random.randint(1,86000)
            dates.append(ptime)
    for i in range(len(dates)):
        astruct[str(dates[i])] = html[atmp[2*i]:atmp[2*i+1]+1]
    order = sorted(float(j) for j in astruct)  # time order
    order.reverse()
    for i in order:
        anw.extend(astruct[str(i)])
    html_nw = html[:atmp[0]]
    html_nw.extend(anw)
    html_nw.extend(html[atmp[-1]+1:])
    return html_nw    

def contentReplace(article):
    nname = '../public_html/markdown-html/about_field.html'  # choose from files in GUI
    ncontent = open(nname,'r',encoding='utf-8').readlines()
    tmp = 0
    clabel = []
    for line in article:
        tmp = tmp + 1
        if 'content label' in line:
            clabel.append(tmp-1)
    print(clabel)
    seg = article[:clabel[0]]
    seg.extend(ncontent[1:-1])
    seg.extend(article[clabel[1]-1:])
    return seg

def modifyContent(html):  # delete or replace certain article
    tmp = 0
    astruct = {}
    atitle = []
    atmp = []
    anw = []
    html_nw = []
    for line in html:
        tmp = tmp + 1
        if 'title-learning' in line:
            atitle.append(html[tmp])
        if 'article label' in line:
            atmp.append(tmp-1)
    atitle_l = [it.lower() for it in atitle]
    for i in range(len(atitle)):
        astruct[atitle_l[i][:-1]] = html[atmp[2*i]-1:atmp[2*i+1]+1]
    print("articles: ", atitle)
    print(astruct.keys())
    choice = input("Delete[D]/Replace[R]: ")
    if choice.lower()=='d':
        name = input("Enter the title of article: ").lower()
        astruct.pop(name)
    elif choice.lower()=='r':
        name = input("Enter the title of article: ").lower()
        astruct[name] = contentReplace(astruct[name])
    for i in astruct.keys():
        anw.extend(astruct[i])
    html_nw = html[:atmp[0]]
    html_nw.extend(anw)
    html_nw.extend(html[atmp[-1]+1:])
    return html_nw 
        


learning_path = '../public_html/LEARNING'
items = os.listdir(learning_path)
for i in range(len(items)):
    items[i] = items[i][:-5]

# delete  ->  manually is okay
dele = 'Fortran'  # choose from GUI
if dele in items:
    items.remove(dele)
    os.remove(learning_path+'/'+dele+'.html')
    print(dele, "has been removed!")
else: # will not happen in GUI
    print(dele," is not in the directory!")


# add file
name = input("Enter the category name: ") # input from GUI
tmp = 0
items_l = [it.lower() for it in items]
if name.lower() in items_l:
    print(name, " is already in the directory!")
else:
    template = open('../public_html/templates/Learning-template.html', 'r', encoding='utf-8').readlines()
    tmp = tmp + 1
    ttmp = 0
    for line in template:
        ttmp = ttmp + 1
        if '<base href' in line:
            template[ttmp-1] = '\t\t<base href="./' + name + '">\n'
        if '<title>' in line:
            posi = findPositions(line)
            template[ttmp-1] = line[:posi[0]] + name.title() + ' - Ian' + line[posi[1]:]
    open(learning_path+'/'+name+'.html', 'w', encoding='utf-8').writelines(template)
    items.append(name)


# add content
page_name = input("Enter page name: ")  # title of adding content
cname = '../public_html/markdown-html/about_observation.html'  # choose from files in GUI
content = open(cname,'r',encoding='utf-8').readlines()

# name = 'Database'  # name of catagory
seg1 = []
seg2 = []
copy_part = []
seg_tmp = 0
insert_position = []
tmp = 0
alabel = '<!-- article label -->\n'
clabel = 'content label'
for line in open(learning_path+'/'+name+'.html', 'r', encoding='utf-8').readlines():
    tmp = tmp + 1
    if alabel in line:
        seg_tmp = seg_tmp + 1
        insert_position.append(tmp)
    if seg_tmp==0:
        seg1.append(line)
    if seg_tmp==1:
        copy_part.append(line)
    if seg_tmp>=2:
        seg2.append(line)
copy_part.append(alabel)
seg2 = seg2[1:]

htmp = 0
for line in seg1:
    htmp = htmp + 1
    if '<base href' in line:
        seg1[htmp-1] = '\t\t<base href="./' + name + '">\n'
            
# modify copy_part here
nw = copy_part[:]
tmp=0
cposi = []
for line in nw:
    tmp = tmp + 1
    if 'title-learning' in line:
        template_check = nw[tmp]
        nw[tmp] = page_name + '\n'
    if 'datetime' in line:
        posi = findPositions(line)
        if input("Use today's date?[Y/N]: ").lower()=='N'.lower():
            edate = input("Enter a date with the form like January 01, 2001: ")
            nw[tmp-1] = line[:posi[0]] + edate + line[posi[1]:]
        else:
            today = date.today()
            nw[tmp-1] = line[:posi[0]] + today.strftime("%B %d, %Y") + line[posi[1]:]
    if 'content label' in line:
        cposi.append(tmp)
nw2 = nw[:cposi[0]]
nw2.extend(content[1:-1])
nw2.extend(nw[cposi[1]-1:])

seg1.extend(nw2)
if 'template' not in template_check:
    seg1.extend(copy_part)
seg1.extend(seg2)

seg = sortbyDate(seg1)

if input("modify? [Y/N]: ").lower()=='y':
    seg_nw = modifyContent(seg)

open(learning_path+'/'+name+'.html', 'w', encoding='utf-8').writelines(seg_nw)

Fortran  is not in the directory!
Enter the category name: idl
idl  is already in the directory!
Enter page name: Fifth
Use today's date?[Y/N]: n
Enter a date with the form like January 01, 2001: August 28, 2019
modify? [Y/N]: n


In [76]:
import os
import random
from datetime import date
from datetime import datetime

def contentReplace(article):
    nname = '../public_html/markdown-html/about_field.html'  # choose from files in GUI
    ncontent = open(nname,'r',encoding='utf-8').readlines()
    tmp = 0
    clabel = []
    for line in article:
        tmp = tmp + 1
        if 'content label' in line:
            clabel.append(tmp-1)
    print(clabel)
    seg = article[:clabel[0]]
    seg.extend(ncontent[1:-1])
    seg.extend(article[clabel[1]-1:])
    return seg

def modifyContent(html):  # delete or replace certain article
    tmp = 0
    astruct = {}
    atitle = []
    atmp = []
    anw = []
    html_nw = []
    for line in html:
        tmp = tmp + 1
        if 'title-learning' in line:
            atitle.append(html[tmp])
        if 'article label' in line:
            atmp.append(tmp-1)
    atitle_l = [it.lower() for it in atitle]
    for i in range(len(atitle)):
        astruct[atitle_l[i][:-1]] = html[atmp[2*i]-1:atmp[2*i+1]+1]
    print("articles: ", astruct.keys())
    choice = input("Delete[D]/Replace[R]: ")
    if choice.lower()=='d':
        name = input("Enter the title of article: ").lower()
        astruct.pop(name)
    elif choice.lower()=='r':
        name = input("Enter the title of article: ").lower()
        astruct[name] = contentReplace(astruct[name])
    for i in astruct.keys():
        anw.extend(astruct[i])
    html_nw = html[:atmp[0]]
    html_nw.extend(anw)
    html_nw.extend(html[atmp[-1]+1:])
    return html_nw 

name = input("Enter the category name: ") # input from GUI

seg = open(learning_path+'/'+name+'.html', 'r', encoding='utf-8').readlines()

if input("modify? [Y/N]: ").lower()=='y':
    seg_nw = modifyContent(seg)

open(learning_path+'/'+name+'.html', 'w', encoding='utf-8').writelines(seg_nw)

Enter the category name: idl
modify? [Y/N]: y
articles:  dict_keys(['fourth', 'third', 'second'])
Delete[D]/Replace[R]: second


In [50]:
# add new essay

from datetime import date

def findPositions(line):
    posi_ini = line.find('<')
    posi1 = line.find('>') + 1
    posi2 = line[posi_ini+1:].find('<') + posi_ini + 1
    return (posi1, posi2)


name = input("Enter name of new essay: ")
essay_path = '../public_html/ESSAY'

lines = open('../public_html/templates/Essay-single-template.html','r',encoding='utf-8').readlines()

tmp = 0
once = 0
cposi = []
for line in lines:
    tmp = tmp + 1
    if '<base href' in line:
            lines[tmp-1] = '\t\t<base href="./' + name + '">\n'
    if '<title>' in line:
        posi = findPositions(line)
        lines[tmp-1] = line[:posi[0]] + name.title() + ' - Ian' + line[posi[1]:]
    if 'POST TITLE' in line:
        lines[tmp+1] = name + '\n'
    if 'datetime' in line:
        posi = findPositions(line)
        if input("Use today's date?[Y/N]: ").lower()=='N'.lower():
            edate = input("Enter a date with the form like January 01, 2001: ")  # choose from GUI can avoid mistake of typing
            lines[tmp-1] = line[:posi[0]] + edate + line[posi[1]:]
        else:
            edate = date.today()
            lines[tmp-1] = line[:posi[0]] + edate.strftime("%B %d, %Y") + line[posi[1]:]
    if 'content label' in line:
        cposi.append(tmp-1)
        once = once + 1
    if once >= 2:
        break

cname = '../public_html/markdown-html/to_Holden.html'  # choose from files in GUI
content = open(cname,'r',encoding='utf-8').readlines()
new = lines[:cposi[0]+1]
new.extend(content[1:-1])
new.extend(lines[cposi[1]:])

open(essay_path+'/'+name+'.html','w',encoding='utf-8').writelines(new)

Enter name of new essay: test3
Use today's date?[Y/N]: y


In [51]:
# configure essay.html

import os
import math
from datetime import datetime

def findPositions(line):
    posi_ini = line.find('<')
    posi1 = line.find('>') + 1
    posi2 = line[posi_ini+1:].find('<') + posi_ini + 1
    return (posi1, posi2)

def titleDate(item):
    essay_path = '../public_html/ESSAY'
    tmp = 0
    once = 0
    pposi = []
    lines = open(essay_path+'/'+item,'r',encoding='utf-8').readlines()
    for line in lines:
        tmp = tmp + 1
        if 'POST TITLE' in line:
            ptitle = lines[tmp+1]
        if 'POST DATE' in line:
            posi = findPositions(lines[tmp+1])
            pdate = lines[tmp+1][posi[0]:posi[1]]
        if 'article label' in line:
            pposi.append(tmp-1)
            once = once + 1
        if once >= 2:
            break
    pcontent = lines[pposi[0]:pposi[1]+1]
    tmp = 0
    for line in pcontent:
        tmp = tmp + 1
        if 'POST TITLE' in line:
            pcontent[tmp+1] = '<a href="ESSAY/' + ptitle.strip() + ' ">' + ptitle.strip() + '</a>\n'
    return (ptitle, pdate, pcontent)
        
            
essay_path = '../public_html/ESSAY'
fpath = '../public/Essay.html'
essays = os.listdir(essay_path)
essays_tit = {}
essays_dat = {}
essays_con = {}
time_array = []
for item in essays:
    [ptitle, pdate, pcontent] = titleDate(item)
    ptime = datetime.strptime(pdate, "%B %d, %Y").timestamp()
    time_array.append(ptime)
    if ptime in time_array:
        ptime = ptime + random.randint(1,86000)
    essays_tit[str(ptime)] = ptitle
    essays_dat[str(ptime)] = pdate
    essays_con[str(ptime)] = pcontent
order = sorted(float(i) for i in essays_tit)  # time order
order.reverse()


# for odr in order:
#     print(essays_tit[str(odr)])

htmls = os.listdir('../public_html')
for html in htmls:
    if 'Essay-page' in html:
        os.remove('../public_html/'+html)

num = 3  # number of essays per page
page_num = math.ceil(len(essays) / num)

template = open('../public_html/Essay-template.html','r',encoding='utf-8').readlines()
tmp = 0
once = 0
alabel = []
for line in template:
    tmp = tmp + 1
    if 'article label' in line:
        alabel.append(tmp-1)
        once = once + 1
    if once>=2:
        break
seg1 = template[:alabel[0]]  # no article label
seg2 = template[alabel[1]+1:]

fname = []
for i in range(1,page_num+1):
    head = seg1[:]
    foot = seg2[:]
    body = []
    if i==1:
        fname.append('Essay.html')
        if page_num==1:
            tmp = 0
            for line in foot:
                tmp = tmp + 1
                if 'NEWER PAGE' in line:
                    foot[tmp] = '\t\t\t\t<span class="disabled next">\n'
                    foot[tmp+1] = '→\n'
                
    elif i==page_num:
        fname.append('Essay-page' + str(i) + '.html')
        tmp = 0
        for line in foot:
            tmp = tmp + 1
            if 'OLDER PAGE' in line:
                foot[tmp] = '\t\t\t\t<span class="prev">\n'
                foot[tmp+1] = '<a href="./' + 'Essay-page' + str(i-1) + '">←</a>\n'
            if 'NEWER PAGE' in line:
                foot[tmp] = '\t\t\t\t<span class="disabled next">\n'
                foot[tmp+1] = '→\n'
    else:
        fname.append('Essay-page' + str(i) + '.html')
        tmp = 0
        for line in foot:
            tmp = tmp + 1
            if 'OLDER PAGE' in line:
                foot[tmp] = '\t\t\t\t<span class="prev">\n'
                if i==2:
                    foot[tmp+1] = '<a href="./Essay">←</a>\n'
                else:
                    foot[tmp+1] = '<a href="./' + 'Essay-page' + str(i-1) + '">←</a>\n'
            if 'NEWER PAGE' in line:
                foot[tmp] = '\t\t\t\t<span class="next">\n'
                foot[tmp+1] = '<a href="./' + 'Essay-page' + str(i+1) + '">→</a>\n'
        
    if i!=page_num or len(essays)%num==0:  # not last page
        for j in range(num*(i-1),num*i):
            body.extend(essays_con[str(order[j])])
    else:
        for j in range(num*(i-1),len(essays)%num+num*(i-1)):
            body.extend(essays_con[str(order[j])])
    htmp = 0
    for line in head:
        htmp = htmp + 1
        if '<base href' in line:
            head[htmp-1] = '\t\t<base href="./' + fname[i-1][:-5] + '">\n'
    head.extend(body)
    head.extend(foot)
    open('../public_html/'+fname[i-1],'w',encoding='utf-8').writelines(head)
    
    

In [ ]:
# configure memories.html

from PIL import Image
import os

def findPositions(line):
    posi_ini = line.find('<')
    posi1 = line.find('>') + 1
    posi2 = line[posi_ini+1:].find('<') + posi_ini + 1
    return (posi1, posi2)

imgs = os.listdir('../public_html/memories')


# add new

img_path = '../imgs/202001.jpg'  # choose from certain directory in GUI
img = Image.open(img_path)
img_re = img.resize((1280,851))
img_name = input("Enter name of the photo(eg. 202001): ")
img_re.save('../public_html/memories/'+img_name+'.jpg', quality=90)
img_title = input("Enter dicription of the photo: ")

lines = open('../public_html/Memories.html','r',encoding='utf-8').readlines()
slabel = []
tlabel = []
circle_label = []
dot_label = []
num_label = []
tmp = 0
for line in lines:
    tmp = tmp + 1
    if 'slide label' in line:
        slabel.append(tmp-1)
    if 'thumbnail label' in line:
        tlabel.append(tmp-1)
    if 'circles label' in line:
        circle_label.append(tmp-1)
    if 'dots label' in line:
        dot_label.append(tmp-1)
    if 'numbers label' in line:
        num_label.append(tmp-1)
img_names = []
img_titles = []
for i in range(len(slabel)//2):
    for line in lines[slabel[2*i]:slabel[2*i+1]]:
        if 'data-src' in line:
            img_names.append(line[24:-2])
        if 'class="title' in line:
            posi = findPositions(line)
            img_titles.append(line[posi[0]:posi[1]])

slidestruc = {}
thumbstruc = {}
titlestruc = {}
tmp = 0
for i in img_names:
    tmp = tmp + 1
    slidestruc[i] = lines[slabel[2*(tmp-1)]:slabel[2*(tmp-1)+1]+1]
    thumbstruc[i] = lines[tlabel[2*(tmp-1)]:tlabel[2*(tmp-1)+1]+1]
    titlestruc[i] = img_titles[tmp-1]

print(imgs)
if input("Do you want to delete one of them? [Y/N]: ").lower()=='y':
    delete = input("Enter the name of the image: ")
    slidestruc.pop(delete+'.jpg')
    thumbstruc.pop(delete+'.jpg')
    titlestruc.pop(delete+'.jpg')
    os.remove('../public_html/memories/'+delete+'.jpg')
    
sbody = lines[slabel[0]:slabel[1]+1]
tbody = lines[tlabel[0]:tlabel[1]+1]
tmp = 0
for line in sbody:
    tmp = tmp + 1
    if 'data-src' in line:
        sbody[tmp-1] = '<img data-src="memories/' + img_name + '.jpg"\n'
    if 'data-image=' in line:
        sbody[tmp-1] = '  data-image="memories/' + img_name + '.jpg"\n'
    if 'jpg">' in line:
        sbody[tmp-1] = '  src="memories/' + img_name + '.jpg">\n'
    if 'class="title"' in line:
        sbody[tmp-1] = '  <p class="title">' + img_title + '</p>\n'
tmp = 0
for line in tbody:
    tmp = tmp + 1
    if 'data-src' in line:
        tbody[tmp-1] = '<img data-src="memories/' + img_name + '.jpg"\n'
    if 'data-image=' in line:
        tbody[tmp-1] = '  data-image="memories/' + img_name + '.jpg"\n'

slidestruc[img_name+'.jpg'] = sbody
thumbstruc[img_name+'.jpg'] = tbody
titlestruc[img_name+'.jpg'] = img_title
order = sorted(slidestruc)
order.reverse()

seg1 = lines[:slabel[0]]
seg2 = []
for item in order:
    seg2.extend(slidestruc[item])  # begin with slide label
seg3 = lines[slabel[-1]+1:circle_label[0]+1]  # begin and end with labels
seg4 = []
seg4.extend('<div class="circles gallery-nav">\n')  # no label
for i in range(len(order)):
    seg4.extend('  <span class="circle"></span>\n')
seg4.extend('</div>\n')
seg5 = lines[circle_label[-1]:dot_label[0]+1]  # begin and end with labels
seg6 = []
seg6.extend('<div class="dots gallery-nav">\n')
for i in range(len(order)):
    seg6.extend('  <span class="dot"></span>\n')
seg6.extend('</div>\n')
seg7 = lines[dot_label[-1]:num_label[0]+1] # begin and end with labels
seg8 = []
seg8.extend('<div class="numbers gallery-nav">\n')
for i in range(len(order)):
    seg8.extend('  <span class="number">' + str(i+1) + '</span>\n')
seg8.extend('</div>\n')
seg9 = lines[num_label[-1]:tlabel[0]]
seg10 = []
for item in order:
    seg10.extend(thumbstruc[item])
seg11 = lines[tlabel[-1]+1:]

seg1.extend(seg2)
seg1.extend(seg3)
seg1.extend(seg4)
seg1.extend(seg5)
seg1.extend(seg6)
seg1.extend(seg7)
seg1.extend(seg8)
seg1.extend(seg9)
seg1.extend(seg10)
seg1.extend(seg11)

open('../public_html/memories.html','w',encoding='utf-8').writelines(seg1)


In [89]:
# configure musing.html

from datetime import datetime
from datetime import date

def findPositions(line):
    posi_ini = line.find('<')
    posi1 = line.find('>') + 1
    posi2 = line[posi_ini+1:].find('<') + posi_ini + 1
    return (posi1, posi2)

lines = open('../public_html/Musings.html','r',encoding='utf-8').readlines()
tmp = 0
plabel = []
for line in lines:
    tmp = tmp + 1
    if 'post label' in line:
        plabel.append(tmp-1)

dates = []
contents = []
for i in range(len(plabel)//2):
    tmp = 0
    for line in lines[plabel[2*i]:plabel[2*i+1]]:
        tmp = tmp + 1
        if 'datetime' in line:
            posi = findPositions(line)
            dates.append(datetime.strptime(line[posi[0]:posi[1]], "%B %d, %Y").timestamp())
        if 'text-align-center' in line:
            contents.append(lines[plabel[2*i]+tmp])

cstruct = {}
sstruct = {}
for i in range(len(dates)):
    cstruct[str(dates[i])] = contents[i]
    sstruct[str(dates[i])] = lines[plabel[2*i]:plabel[2*i+1]+1]

# delete
print(cstruct)
if input("Do you want to delete on of them?[Y/N]").lower()=='y':
    delete = input("Enter the date: ")
    cstruct.pop(delete)
    sstruct.pop(delete)

# add
mbody = lines[plabel[0]:plabel[1]+1]
tmp = 0
for line in mbody:
    tmp = tmp + 1
    if 'datetime' in line:
        if input("Use today's date?[Y/N]: ").lower()=='n':
            mdate = input("Enter a date with the form like January 01, 2001: ")  # choose from GUI can avoid mistake of typing
        else:
            mdate_tmp = date.today()
            mdate = mdate_tmp.strftime("%B %d, %Y")
        mbody[tmp-1] = '    <time datetime="2021-06-07">' + mdate + '</time>\n'
    if 'text-align-center' in line:
        musing = input("Input your musing: ")
        mbody[tmp] = musing
date_num = datetime.strptime(mdate, "%B %d, %Y").timestamp()
cstruct[str(date_num)] = musing
sstruct[str(date_num)] = mbody

order = sorted([float(i) for i in sstruct])
order.reverse()

body = []
for item in order:
    body.extend(sstruct[str(item)])

head = lines[:plabel[0]]
foot = lines[plabel[-1]+1:]

head.extend(body)
head.extend(foot)


open('../public_html/Musings.html','w',encoding='utf-8').writelines(head)

{'1513522800.0': '做什么都太毛躁了，放慢脚步please。\n'}
Do you want to delete on of them?[Y/N]n
Use today's date?[Y/N]: n
Enter a date with the form like January 01, 2001: April 11, 2018
Input your musing: 巴萨输球了，决定写一个关于足球的故事。 对于强电，是时候坦然接受了吧！


In [38]:
from datetime import datetime

for i in ['1299423600.0', '1622991600.0', '1624719600.0', '992703600.0']:
    print(datetime.fromtimestamp(float(i)))

2011-03-07 00:00:00
2021-06-07 00:00:00
2021-06-27 00:00:00
2001-06-17 00:00:00


In [43]:
te = ['1299423600.0', '1622991600.0', '1624719600.0', '992703600.0']
kk = sorted([float(i) for i in te])
print(kk)

[992703600.0, 1299423600.0, 1622991600.0, 1624719600.0]


In [37]:
from datetime import datetime

stst = ['June 23, 2021', 'June 25, 2021' ]
kk = []

for i in stst:
    tt = datetime.strptime(i, "%B %d, %Y")
    kk.append(tt.timestamp())
    
kk[0] - kk[1]

-172800.0

In [57]:
aa = {}
aa['tsho'] = 1234
aa['asdf'] = 'asdfqw'

for i in aa.keys():
    print(aa[i])

1234
asdfqw
